# 서울시 공공자전거 이용정보(22년도 전체)

In [111]:
import glob
import pandas as pd
import geopy.distance
import geopandas as gpd

In [13]:
a = pd.read_csv('../ddarung_data/서울특별시 공공자전거 이용정보(22년도 전체)_2.csv', encoding = 'utf8')
a.head()

,Unnamed: 0,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분)
0,1000000,202209,1738,1738. 창동주공17단지 상가앞,일일권,M,60대,3,139.21,1.25,5408.04,60
1,1000001,202209,1738,1738. 창동주공17단지 상가앞,일일권,M,~10대,19,1681.96,15.26,65682.08,504
2,1000002,202209,1738,1738. 창동주공17단지 상가앞,일일권,M,기타,10,1573.06,12.85,55328.58,218
3,1000003,202209,1738,1738. 창동주공17단지 상가앞,일일권(비회원),NaN,기타,13,1014.8,9.17,39425.45,428
4,1000004,202209,1738,1738. 창동주공17단지 상가앞,일일권(비회원),F,기타,1,75.68,0.68,2940.00,39


In [3]:
a.shape[0]

1383448

# 테스트

In [132]:
ddarung = pd.read_csv('../ddarung_master.csv', encoding = 'cp949')
print(ddarung.shape)

ddarung['대여소_ID'] = ""
ddarung['주소1'] = ""
ddarung['주소2'] = ""
ddarung['ID_distance'] = ""
ddarung.head()

(2719, 17)


,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter,public_place_in_100m,...,shortest_bus_meter,public_transport_in_100m,schools_in_100m,unis_in_100m,total_schools_in_100m,public_places_and_schools_in_100m,대여소_ID,주소1,주소2,ID_distance
0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,내자동,경복궁,183.7,0,...,6.2,5,0,0,0,0,,,,
1,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,사직로,적선동,경복궁,51.1,0,...,76.2,3,0,0,0,0,,,,
2,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,세종대로23길,당주동,광화문,195.5,0,...,161.8,0,0,0,0,0,,,,
3,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978332,종로1길,세종로,광화문,224.9,0,...,95.8,1,0,0,0,0,,,,
4,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,경희궁1길,신문로2가,광화문,505.0,3,...,13.3,4,0,0,0,3,,,,


In [119]:
ddarung[ddarung['위도'] == '37.53439*']

,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter,public_place_in_100m,shortest_bus,shortest_bus_meter,public_transport_in_100m,schools_in_100m,unis_in_100m,total_schools_in_100m,public_places_and_schools_in_100m,대여소_ID,주소1,주소2


In [142]:
master = pd.read_csv("../ddarung_data/서울시 따릉이대여소 마스터 정보.csv", encoding = 'cp949')
master= master[master['위도'] != 0]
master= master[master['경도'] != 0]
master_gdf = gpd.GeoDataFrame(
    master, geometry=gpd.points_from_xy(master['위도'], master['경도']), crs="EPSG:4326"
)

print(master_gdf.shape)
master_gdf.head()

(3146, 6)


,대여소_ID,주소1,주소2,위도,경도,geometry
2,ST-997,서울특별시 양천구 목동중앙로 49,목동3단지 시내버스정류장,37.534390,126.869598,POINT (37.53439 126.86960)
3,ST-996,서울특별시 양천구 남부순환로88길5-16,양강중학교앞 교차로,37.524334,126.850548,POINT (37.52433 126.85055)
4,ST-995,서울특별시 양천구 신정동 621-14,계남공원 입구 주출입구 좌측,37.510681,126.857399,POINT (37.51068 126.85740)
5,ST-994,서울특별시 양천구 목동서로161,SBS방송국,37.529163,126.872749,POINT (37.52916 126.87275)
6,ST-993,서울특별시 양천구 신월로 342-1 구두수선대19,NaN,37.521511,126.857384,POINT (37.52151 126.85738)


In [145]:
distance = 100
for index, row in ddarung.iterrows():
    for index2, row2 in master_gdf.iterrows():
        dist = geopy.distance.geodesic((row['위도'], row['경도']),(row2['geometry'].x, row2['geometry'].y))
        if(dist.meters <= distance):
            distance = round(dist.meters,3)
            ddarung['대여소_ID'][index] = row2['대여소_ID']
            ddarung['주소1'][index] = row2['주소1']
            ddarung['주소2'][index] = row2['주소2']
            ddarung['ID_distance'][index] = distance
    
    print(index , " " , row['보관소(대여소)명'], " : " , ddarung['대여소_ID'][index], " : ", ddarung['주소1'][index], " : ", distance)
    print(index , "/", ddarung.shape[0], " " , round(index / ddarung.shape[0] * 100, 2), "%")
    
    distance = 100


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

0    경복궁역 7번출구 앞  :  ST-117  :  서울특별시 종로구 사직로 지하130  :  0.03
0 / 2719   0.0 %
1    경복궁역 4번출구 뒤  :  ST-118  :  서울특별시 종로구 사직로 지하130  :  0.023
1 / 2719   0.04 %
2    광화문역 1번출구 앞  :  ST-119  :  서울특별시 종로구 세종대로 지하189  :  0.037
2 / 2719   0.07 %
3    종로구청 옆  :  ST-121  :  서울특별시 종로구 세종로 84-1  :  0.06
3 / 2719   0.11 %
4    서울역사박물관 앞  :  ST-123  :  서울특별시 종로구 새문안로 55  :  0.036
4 / 2719   0.15 %
5    광화문 S타워 앞  :  ST-124  :  서울특별시 종로구 새문안로 82  :  0.02
5 / 2719   0.18 %
6    광화문역 6번출구 옆 B  :  ST-125  :  서울특별시 종로구 새문안로 102-2 가로판매대  :  0.051
6 / 2719   0.22 %
7    국립현대미술관  :  ST-130  :  서울특별시 종로구 삼청로 30  :  0.014
7 / 2719   0.26 %
8    종각역 1번출구 앞  :  ST-132  :  서울특별시 종로구 종로 47  :  0.058
8 / 2719   0.29 %
9    안국역 5번출구 앞  :  ST-142  :  서울특별시 종로구 율곡로 지하62  :  0.058
9 / 2719   0.33 %
10    낙원상가 옆  :  ST-143  :  서울특별시 종로구 삼일대로 428  :  0.042
10 / 2719   0.37 %
11    탑골공원 앞  :  ST-144  :  서울특별시 종로구 종로 99  :  0.054
11 / 2719   0.4 %
12    청계2가 사거리 옆  :  ST-145  :  서울특별시 종로구 청계천로 93 파고다타워  :  0.053
12 / 271

103    종로문화원 건너편  :  ST-3090  :  서울특별시 종로구 중학동 10-3  :  0.015
103 / 2719   3.79 %
104    정동사거리  :  ST-116  :  서울특별시 중구 정동길 2-1  :  0.05
104 / 2719   3.82 %
105    청계광장 옆  :  ST-126  :  서울특별시 중구 세종대로 136  :  0.04
105 / 2719   3.86 %
106    광교사거리 남측  :  ST-134  :  서울특별시 중구 남대문로 125 DGB금융센터  :  0.009
106 / 2719   3.9 %
107    을지로입구역 4번출구 앞  :  ST-136  :  서울특별시 중구 을지로 55  :  0.024
107 / 2719   3.94 %
108    KEB 하나금융그룹 명동사옥 옆  :  ST-137  :  서울특별시 중구 을지로 66  :  0.044
108 / 2719   3.97 %
109    명동성당 앞  :  ST-138  :  서울특별시 중구 명동길 74  :  0.001
109 / 2719   4.01 %
110    신세계백화점 본점 앞  :  ST-140  :  서울특별시 중구 충무로1가 53-6  :  0.038
110 / 2719   4.05 %
111    을지로2가 사거리 북측  :  ST-147  :  서울특별시 중구 을지로 79  :  0.06
111 / 2719   4.08 %
112    을지로2가 사거리 남측  :  ST-148  :  서울특별시 중구 을지로 100  :  0.05
112 / 2719   4.12 %
113    충무로역 6번출구 앞  :  ST-152  :  서울특별시 중구 퇴계로 214 충무로역 3.4호선  :  0.031
113 / 2719   4.16 %
114    맥스타일 앞  :  ST-162  :  서울특별시 중구 마장로 3  :  0.04
114 / 2719   4.19 %
115    동대문역사문화공원역 9번출구 앞  :  

202    삼각지역 3번출구  :  ST-334  :  서울특별시 용산구 한강로1가 228-1  :  0.022
202 / 2719   7.43 %
203    LIG강촌아파트 103동앞  :  ST-335  :  서울특별시 용산구 이촌동301-5  :  0.013
203 / 2719   7.47 %
204    신용산역 6번출구 앞  :  ST-435  :  서울특별시 용산구 한강대로 115  :  0.05
204 / 2719   7.5 %
205    삼각지역 4번출구 앞  :  ST-436  :  서울특별시 용산구 한강대로 161  :  0.013
205 / 2719   7.54 %
206    숙명여대 입구 교차로  :  ST-437  :  서울특별시 용산구 청파로47길 100  :  0.026
206 / 2719   7.58 %
207    선린인터넷 고등학교  :  ST-438  :  서울특별시 용산구 원효로97길 33-4  :  0.04
207 / 2719   7.61 %
208    청파동입구 교차로  :  ST-439  :  서울특별시 용산구 한강대로 349  :  0.061
208 / 2719   7.65 %
209    이촌1동 마을공원  :  ST-440  :  서울특별시 용산구 한강로3가 66-4  :  0.045
209 / 2719   7.69 %
210    효창동주민센터 앞  :  ST-441  :  서울특별시 용산구 효창원로 161  :  0.033
210 / 2719   7.72 %
211    서빙고동 주민센터 앞  :  ST-442  :  서울특별시 용산구 서빙고로 279  :  0.031
211 / 2719   7.76 %
212    국군복지단  :  ST-1324  :  서울특별시 용산구 두텁바위로 54-99  :  0.042
212 / 2719   7.8 %
213    숙대입구역 8번  :  ST-1325  :  서울특별시 용산구 한강대로 289  :  0.021
213 / 2719   7.83 %
214    베

299    성수사거리 버스정류장 앞  :  ST-112  :  서울특별시 성동구 아차산로 171  :  0.047
299 / 2719   11.0 %
300    청계천 박물관 앞  :  ST-234  :  서울특별시 성동구 마장동 610  :  0.059
300 / 2719   11.03 %
301    양지사거리  :  ST-235  :  서울특별시 성동구 홍익동 590  :  0.033
301 / 2719   11.07 %
302    상왕십리역 4번 출구 앞  :  ST-236  :  서울특별시 성동구 하왕십리동 960-3  :  0.018
302 / 2719   11.11 %
303    왕십리역 11번 출구 앞  :  ST-237  :  서울특별시 성동구 행당동 188-9  :  0.031
303 / 2719   11.14 %
304    금호역 1번출구 앞  :  ST-238  :  서울특별시 성동구 금호동3가 1575-3  :  0.024
304 / 2719   11.18 %
305    옥수동성당 옆  :  ST-239  :  서울특별시 성동구 옥수동 산5-148  :  0.002
305 / 2719   11.22 %
306    래미안금호하이리버 아파트 102동 옆  :  ST-240  :  서울특별시 성동구 금호동2가 1220-1  :  0.037
306 / 2719   11.25 %
307    한양대병원사거리  :  ST-241  :  서울특별시 성동구 행당동 20-2  :  0.011
307 / 2719   11.29 %
308    용답토속공원 앞  :  ST-242  :  서울특별시 성동구 용답동 224-1  :  0.037
308 / 2719   11.33 %
309    장한평역 8번 출구 앞  :  ST-243  :  서울특별시 성동구 용답동 224-3  :  0.031
309 / 2719   11.36 %
310    청계벽산아파트 앞  :  ST-244  :  서울특별시 성동구 하왕십리동 286-24  :  0.053
3

...

2285    삼성동베이직하우스앞  :  ST-1245  :  서울특별시 강남구 봉은사로 456  :  0.018
2285 / 2719   84.04 %
2286    언주역 6번출구앞  :  ST-1246  :  서울특별시 강남구 논현로 566  :  0.03
2286 / 2719   84.08 %
2287    역삼동 sk뷰 501동앞  :  ST-1247  :  서울특별시 강남구 역삼로 315-1  :  0.044
2287 / 2719   84.11 %
2288    보금자리정원  :  ST-1248  :  서울특별시 강남구 헌릉로569길 39-6  :  0.041
2288 / 2719   84.15 %
2289    자곡사거리  :  ST-1364  :  서울특별시 강남구 밤고개로 206  :  0.052
2289 / 2719   84.19 %
2290    학동역  :  ST-1365  :  서울특별시 강남구 학동로 지하 180  :  0.053
2290 / 2719   84.22 %
2291    크린랩 본사  :  ST-1366  :  서울특별시 강남구 봉은사로 지하 201  :  0.041
2291 / 2719   84.26 %
2292    래미안강남힐즈 사거리  :  ST-1407  :  서울특별시 강남구 자곡로 101  :  0.008
2292 / 2719   84.3 %
2293    강남구 도시관리공단  :  ST-1433  :  서울특별시 강남구 삼성로 628  :  0.038
2293 / 2719   84.33 %
2294    경기여자고등학교 후문 (삼성로3길 입구)  :  ST-1559  :  서울특별시 강남구 개포로 지하 420  :  0.032
2294 / 2719   84.37 %
2295    구룡마을 입구 (래미안블레스티지 아파트)  :  ST-1560  :  서울특별시 강남구 양재대로 478  :  0.039
2295 / 2719   84.41 %
2296    구룡터널 입구(개포1단지아파트)  :  ST-1561  :

2377    레베쌍트 빌딩 앞  :  ST-3111  :  서울특별시 강남구 봉은사로86길 6 빌딩 레베쌍트  :  0.042
2377 / 2719   87.42 %
2378    도심공항타워 앞  :  ST-3112  :  서울특별시 강남구 테헤란로87길 36 도심공항타워  :  0.043
2378 / 2719   87.46 %
2379    영동중앙교회 버스정류장  :  ST-3115  :  서울특별시 강남구 논현로 420  :  0.024
2379 / 2719   87.5 %
2380    청담역 1번출구  :  ST-3164  :  서울특별시 강남구 영동대로 646 위레벤646  :  0.032
2380 / 2719   87.53 %
2381   일원에코파크 주차장  :  ST-3179  :  서울특별시 강남구 일원동 698  :  0.046
2381 / 2719   87.57 %
2382   매봉역 1번출구  :  ST-3185  :  서울특별시 강남구 남부순환로 2741  :  0.053
2382 / 2719   87.61 %
2383   학여울역사거리(은마아파트)  :  ST-3178  :  서울특별시 강남구 대치동 398  :  0.045
2383 / 2719   87.64 %
2384    가락시장역 3번 출구  :  ST-838  :  서울특별시 송파구 송파대로 지하 257  :  0.057
2384 / 2719   87.68 %
2385    밀리아나2빌딩 앞  :  ST-737  :  서울특별시 송파구 송파대로28길 24  :  0.048
2385 / 2719   87.72 %
2386    거여역 3번출구  :  ST-930  :  서울특별시 송파구 오금로 지하 499  :  0.053
2386 / 2719   87.75 %
2387    종합운동장역 4번출구  :  ST-539  :  서울특별시 송파구 올림픽로 8  :  0.042
2387 / 2719   87.79 %
2388    9호선종합운동장역 9번출구  :  ST-540  

2472    테라타워2  :  ST-1707  :  서울특별시 송파구 송파대로 201  :  0.013
2472 / 2719   90.92 %
2473    거여동 사거리  :  ST-1716  :  서울특별시 송파구 거여동 11-3  :  0.054
2473 / 2719   90.95 %
2474    가락삼환아파트 앞  :  ST-1717  :  서울특별시 송파구 오금로 지하 402  :  0.054
2474 / 2719   90.99 %
2475    삼전역 1번출구  :  ST-1728  :  서울특별시 송파구 백제고분로 187  :  0.039
2475 / 2719   91.03 %
2476    석촌고분역 4번출구  :  ST-1729  :  서울특별시 송파구 백제고분로 272  :  0.049
2476 / 2719   91.06 %
2477    송파나루역 4번 출구옆  :  ST-1730  :  서울특별시 송파구 오금로 162  :  0.038
2477 / 2719   91.1 %
2478    한성백제역 2번 출구  :  ST-1719  :  서울특별시 송파구 위례성대로 51  :  0.036
2478 / 2719   91.14 %
2479    올림픽공원역 3번출구  :  ST-1720  :  서울특별시 송파구 양재대로 지하 1233  :  0.044
2479 / 2719   91.17 %
2480   석촌동 주민센터  :  ST-1851  :  서울특별시 송파구 백제고분로37길 16 석촌동주민센터  :  0.018
2480 / 2719   91.21 %
2481   가락종합 사회복지관  :  ST-1852  :  서울특별시 송파구 가락동 282  :  0.042
2481 / 2719   91.25 %
2482   경찰병원역 1,2번 출구 사이  :  ST-1853  :  서울특별시 송파구 송이로 123 국립경찰병원  :  0.042
2482 / 2719   91.28 %
2483   문정시영아파트 5동 앞 공원  :  ST-1854  : 

2566    위례포레샤인13단지  :  ST-2917  :  서울특별시 송파구 거여동 657  :  0.056
2566 / 2719   94.37 %
2567    삼전동주민센터 앞  :  ST-2918  :  서울특별시 송파구 백제고분로 236-2 구두수선대  :  0.049
2567 / 2719   94.41 %
2568    마천청소년센터  :  ST-2936  :  서울특별시 송파구 송파구 마천동 325-29  :  0.033
2568 / 2719   94.45 %
2569    송파청소년센터  :  ST-2957  :  서울특별시 송파구 중대로 24 올림픽훼밀리타운  :  0.047
2569 / 2719   94.48 %
2570    거여동사거리 송파둘레길 입구  :  ST-2937  :  서울특별시 송파구 거여동 산23-34  :  0.054
2570 / 2719   94.52 %
2571    위례포레샤인18단지 1802동  :  ST-2938  :  서울특별시 송파구 송파구 거여동 산60-1  :  0.062
2571 / 2719   94.56 %
2572    위례포레샤인18단지 1812동  :  ST-2939  :  서울특별시 송파구 송파구 거여동 657-12  :  0.036
2572 / 2719   94.59 %
2573    H비즈니스파크  :  ST-2984  :  서울특별시 송파구 송파구 문정동 373  :  0.05
2573 / 2719   94.63 %
2574    가든파이브 리빙관  :  ST-2985  :  서울특별시 송파구 충민로 66 가든파이브라이프  :  0.049
2574 / 2719   94.67 %
2575    배명고등학교 건너편  :  ST-2986  :  서울특별시 송파구 송파구 석촌동 302-1  :  0.048
2575 / 2719   94.7 %
2576    호돌이상가  :  ST-2987  :  서울특별시 송파구 중대로 24 올림픽훼밀리타운  :  0.045
2576 / 2719   94.74 %

2658   한영중고앞사거리  :  ST-1766  :  서울특별시 강동구 상일동 168  :  0.038
2658 / 2719   97.76 %
2659   명일파출소  :  ST-1767  :  서울특별시 강동구 명일동 47-20  :  0.014
2659 / 2719   97.79 %
2660   강동경희대학교병원 입구  :  ST-1830  :  서울특별시 강동구 동남로 888  :  0.028
2660 / 2719   97.83 %
2661   (시립)고덕평생학습관  :  ST-1831  :  서울특별시 강동구 고덕로 295 고덕평생학습관  :  0.029
2661 / 2719   97.87 %
2662   명일근린공원공동체텃밭  :  ST-1832  :  서울특별시 강동구 상일동 145-6  :  0.01
2662 / 2719   97.9 %
2663   고덕역 5번출구  :  ST-1833  :  서울특별시 강동구 고덕로 272 헌혈의집  :  0.027
2663 / 2719   97.94 %
2664    SSTS 몰 앞  :  ST-1834  :  서울특별시 강동구 천호동 55-26  :  0.044
2664 / 2719   97.98 %
2665   대명초교입구 사거리  :  ST-1835  :  서울특별시 강동구 명일동 59  :  0.031
2665 / 2719   98.01 %
2666   천동초교 삼거리  :  ST-1836  :  서울특별시 강동구 천호동 114-4  :  0.028
2666 / 2719   98.05 %
2667    천동초교입구사거리 버스정류장(십자성마을 방면)  :  ST-1837  :  서울특별시 강동구 양재대로 1504  :  0.011
2667 / 2719   98.09 %
2668   강동역 1번출구 앞  :  ST-1838  :  서울특별시 강동구 천호동 448-1  :  10.829
2668 / 2719   98.12 %
2669   둔촌사거리  :  ST-1839  :  서울특별시 강동구 성내동 44

In [159]:

distance = 1000
for index2, row2 in master_gdf.iterrows():
    dist = geopy.distance.geodesic((37.505463, 127.077347),(row2['geometry'].x, row2['geometry'].y))
    if(dist.meters <= distance):
        distance = round(dist.meters,3)
        ddarung['대여소_ID'][2422] = row2['대여소_ID']
        ddarung['주소1'][2422] = row2['주소1']
        ddarung['주소2'][2422] = row2['주소2']
        ddarung['ID_distance'][2422] = distance



distance = 1000
ddarung[ddarung['대여소_ID'] == ""]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter,public_place_in_100m,...,shortest_bus_meter,public_transport_in_100m,schools_in_100m,unis_in_100m,total_schools_in_100m,public_places_and_schools_in_100m,대여소_ID,주소1,주소2,ID_distance


In [160]:
ddarung.loc[2422]

보관소(대여소)명                                                 아주중학교건너편
자치구                                                            송파구
상세주소                                             서울특별시 송파구 도곡로 446
위도                                                       37.505463
경도                                                      127.077347
shortest_road                                               도곡로62길
법정동                                                            잠실동
shortest_subway                                              종합운동장
shortest_subway_meter                                        696.1
public_place_in_100m                                             0
shortest_bus                                                 아주중학교
shortest_bus_meter                                            49.5
public_transport_in_100m                                         2
schools_in_100m                                                  0
unis_in_100m                                                  

In [152]:
print(index)

2718


In [150]:
ddarung[ddarung['대여소_ID'] == ""]

,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter,public_place_in_100m,...,shortest_bus_meter,public_transport_in_100m,schools_in_100m,unis_in_100m,total_schools_in_100m,public_places_and_schools_in_100m,대여소_ID,주소1,주소2,ID_distance
748,꿈의숲 롯데캐슬,강북구,서울특별시 강북구 오현로 56 꿈의숲 롯데캐슬 111동 인근,37.618641,127.036240,오패산로30가길,미아동,미아사거리,806.7,0,...,27.1,4,0,0,0,0,,,,
2422,아주중학교건너편,송파구,서울특별시 송파구 도곡로 446,37.505463,127.077347,도곡로62길,잠실동,종합운동장,696.1,0,...,49.5,2,0,0,0,0,,,,


In [161]:
ddarung['대여소_ID'].value_counts()

ST-117     1
ST-304     1
ST-296     1
ST-297     1
ST-298     1
          ..
ST-1475    1
ST-1476    1
ST-1639    1
ST-1640    1
ST-3147    1
Name: 대여소_ID, Length: 2719, dtype: int64

In [162]:
ddarung.to_csv("../ddarung_master.csv", encoding = 'cp949')

In [140]:
ddarung

,보관소(대여소)명,자치구,상세주소,위도,경도,shortest_road,법정동,shortest_subway,shortest_subway_meter,public_place_in_100m,...,shortest_bus_meter,public_transport_in_100m,schools_in_100m,unis_in_100m,total_schools_in_100m,public_places_and_schools_in_100m,대여소_ID,주소1,주소2,ID_distance
0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,사직로12길,내자동,경복궁,183.7,0,...,6.2,5,0,0,0,0,ST-117,서울특별시 종로구 사직로 지하130,경복궁역 7번출구 앞,0.03
1,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,사직로,적선동,경복궁,51.1,0,...,76.2,3,0,0,0,0,ST-118,서울특별시 종로구 사직로 지하130,경복궁역 4번출구 뒤,0.023
2,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,세종대로23길,당주동,광화문,195.5,0,...,161.8,0,0,0,0,0,ST-119,서울특별시 종로구 세종대로 지하189,세종로공원,0.036717
3,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978332,종로1길,세종로,광화문,224.9,0,...,95.8,1,0,0,0,0,ST-121,서울특별시 종로구 세종로 84-1,NaN,0.05956
4,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,경희궁1길,신문로2가,광화문,505.0,3,...,13.3,4,0,0,0,3,ST-123,서울특별시 종로구 새문안로 55,서울역사박물관 앞,0.035522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2714,명일역 3번출구 옆,강동구,강동구 명일동 303-7,37.551426,127.144211,양재대로,명일동,명일,19.7,2,...,29.0,5,0,0,0,2,,,,
2715,상일파출소 옆,강동구,강동구 상일동 520-1,37.556553,127.165878,고덕로79길,상일동,상일동,50.8,0,...,50.9,3,0,0,0,0,,,,
2716,강변그대가리버뷰 입구 맞은편,강동구,강동구 천호동 250-7,37.548164,127.122574,선사로,천호동,암사,495.8,0,...,382.6,0,0,0,0,0,,,,
2717,방아삼거리,강동구,강동구 둔촌동 118-2,37.523281,127.146133,동남로43길,둔촌동,둔촌동,1006.9,0,...,159.8,0,0,0,0,0,,,,


# Seperate

In [11]:
count = 1

for i in range(0, a.shape[0], 1000000):
    
    if(a.shape[0] > i+1000000):
        temp = a[i : i+1000000]
    else:
        temp = a[i : a.shape[0]]
    url = '../ddarung_data/서울특별시 공공자전거 이용정보(22년도 전체)_%d.csv'%count
    temp.to_csv(url, encoding = 'utf8')
    count = count + 1

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
all_files = glob.glob('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202212/*.csv')
print(len(all_files))
all_files_data = []

for file in all_files:
    print(file)
    if(file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202212/tpss_bcycl_od_statnhm_20221107.csv'):
        data_frame = pd.read_csv(file, encoding = 'UTF8')
    else:
        data_frame = pd.read_csv(file, encoding = 'euc-kr')
    all_files_data.append(data_frame)
    
all_files_data_concat = pd.concat(all_files_data, axis = 0, ignore_index = True)
print(all_files_data_concat.shape)
all_files_data_concat.to_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202212.csv', encoding = 'euc-kr')

In [70]:
janurary = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202205.csv', encoding = 'euc-kr')
print(janurary.shape)
janurary.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3,4,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(9290665, 12)


,Unnamed: 0,기준_날짜,기준_시간대,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리,기준_시간,시작_대여소,종료_대여소,전체건수
0,0,20220517,0.0,ST-1017,ST-1495,1.0,4,640,NaN,NaN,NaN,NaN
1,1,20220517,0.0,ST-1019,ST-282,1.0,6,1106,NaN,NaN,NaN,NaN
2,2,20220517,0.0,ST-1020,ST-479,1.0,6,1147,NaN,NaN,NaN,NaN
3,3,20220517,0.0,ST-1025,ST-465,1.0,5,670,NaN,NaN,NaN,NaN
4,4,20220517,0.0,ST-1041,ST-1041,1.0,21,180,NaN,NaN,NaN,NaN


In [73]:
janurary[janurary['종료_대여소_ID'] == 'ST-3019']

,Unnamed: 0,기준_날짜,기준_시간대,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리,기준_시간,시작_대여소,종료_대여소,전체건수
3156,3156,20220517,30.0,ST-2975,ST-3019,1.0,6,916,NaN,NaN,NaN,NaN
3837,3837,20220517,35.0,ST-2975,ST-3019,1.0,6,916,NaN,NaN,NaN,NaN
9691,9691,20220517,155.0,ST-2839,ST-3019,1.0,75,15029,NaN,NaN,NaN,NaN
12995,12995,20220517,310.0,ST-2839,ST-3019,1.0,75,15029,NaN,NaN,NaN,NaN
13053,13053,20220517,315.0,ST-2308,ST-3019,1.0,78,16930,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8594077,8594077,20220525,2000.0,ST-422,ST-3019,1.0,27,3695,NaN,NaN,NaN,NaN
8594992,8594992,20220525,2005.0,ST-2146,ST-3019,1.0,60,3240,NaN,NaN,NaN,NaN
8597790,8597790,20220525,2030.0,ST-422,ST-3019,1.0,27,3695,NaN,NaN,NaN,NaN
8599177,8599177,20220525,2050.0,ST-19,ST-3019,1.0,20,900,NaN,NaN,NaN,NaN


In [18]:
february = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202202.csv', encoding = 'euc-kr')
print(february.shape)
february.head()

(2620331, 8)


,Unnamed: 0,기준_날짜,기준_시간대,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리
0,0,20220212,0,ST-1017,ST-2764,1,8,0
1,1,20220212,0,ST-1061,ST-1845,1,6,0
2,2,20220212,0,ST-1079,ST-892,1,3,620
3,3,20220212,0,ST-1092,ST-502,1,13,1340
4,4,20220212,0,ST-1107,ST-1104,1,11,2205


In [21]:
march = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202203.csv', encoding = 'euc-kr')
print(march.shape)
march.head()

(4690052, 8)


,Unnamed: 0,기준_날짜,기준_시간대,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리
0,0,20220305,0,ST-1034,ST-2210,1,23,2770
1,1,20220305,0,ST-1035,ST-1482,1,7,1250
2,2,20220305,0,ST-1062,ST-1062,1,0,0
3,3,20220305,0,ST-1062,ST-2036,1,10,1630
4,4,20220305,0,ST-1065,ST-527,1,16,2626


In [25]:
april = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202204.csv', encoding = 'euc-kr')
print(april.shape)
april.head()

(7492307, 8)


,Unnamed: 0,기준_날짜,기준_시간대,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리
0,0,20220428,0,ST-1003,ST-2992,1,6,1107
1,1,20220428,0,ST-1005,ST-2005,1,3,575
2,2,20220428,0,ST-1010,ST-2556,1,9,1550
3,3,20220428,0,ST-1015,ST-2844,1,6,1300
4,4,20220428,0,ST-1034,ST-345,1,6,757


In [26]:
may = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202205.csv', encoding = 'euc-kr')
print(may.shape)
may.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3,4,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(9290665, 12)


,Unnamed: 0,기준_날짜,기준_시간대,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리,기준_시간,시작_대여소,종료_대여소,전체건수
0,0,20220517,0.0,ST-1017,ST-1495,1.0,4,640,NaN,NaN,NaN,NaN
1,1,20220517,0.0,ST-1019,ST-282,1.0,6,1106,NaN,NaN,NaN,NaN
2,2,20220517,0.0,ST-1020,ST-479,1.0,6,1147,NaN,NaN,NaN,NaN
3,3,20220517,0.0,ST-1025,ST-465,1.0,5,670,NaN,NaN,NaN,NaN
4,4,20220517,0.0,ST-1041,ST-1041,1.0,21,180,NaN,NaN,NaN,NaN


In [27]:
june = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202206.csv', encoding = 'euc-kr')
print(june.shape)
june.head()

(8062071, 9)


,Unnamed: 0,기준_날짜,기준_시간,시작_대여소,종료_대여소,전체건수,전체_이용_분,전체_이용_거리,전 체건수
0,0,20220602,0,ST-2264,ST-33,1.0,20,2400,NaN
1,1,20220602,0,ST-1875,ST-1046,1.0,23,3667,NaN
2,2,20220602,0,ST-2843,ST-1452,1.0,64,1898,NaN
3,3,20220602,0,ST-1733,ST-535,1.0,36,4880,NaN
4,4,20220602,0,ST-542,ST-983,1.0,35,6418,NaN


In [28]:
july = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202207.csv', encoding = 'euc-kr')
print(july.shape)
july.head()

(3715371, 8)


,Unnamed: 0,기준_날짜,기준_시간,시작_대여소,종료_대여소,전체건수,전체_이용_분,전체_이용_거리
0,0,20220715,0,ST-737,ST-1869,1,10,1836
1,1,20220715,0,ST-235,ST-1780,1,12,2490
2,2,20220715,0,ST-615,ST-2716,1,17,5467
3,3,20220715,0,ST-1781,ST-1781,1,16,1340
4,4,20220715,0,ST-502,ST-1041,1,6,920


In [29]:
august = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202208.csv', encoding = 'utf8')
print(august.shape)
august.head()

(6903282, 10)


,20220801,0,0000,ST-2921,상봉1동_020_3,ST-2921.1,상봉1동_020_3.1,1,55,7842.41
0,20220801,0,0,ST-1909,반포3동_035_2,ST-1922,양재1동_012_1,1,35,5269.87
1,20220801,0,0,ST-2637,가산동_035_2,ST-3032,독산1동_037_1,1,8,940.00
2,20220801,0,0,ST-1609,숭인2동_005_1,ST-272,용신동_050_1,1,14,1263.72
3,20220801,0,0,ST-400,청량리동_040_1,ST-2267,청량리동_029_1,1,7,713.53
4,20220801,0,0,ST-1081,송파1동_038_1,ST-1729,삼전동_046_1,1,21,1967.57


In [30]:
september = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202209.csv', encoding = 'utf8')
print(september.shape)
september.head()

(9000600, 10)


,20220901,0,0000,ST-910,월곡2동_015_1,ST-2935,안암동_024_1,1,14,2405
0,20220901,0,0,ST-600,월계1동_023_1,ST-2714,월계1동_015_1,1,4,1060.00
1,20220901,0,0,ST-827,안암동_030_1,ST-1215,안암동_023_1,1,3,492.15
2,20220901,0,0,ST-253,군자동_038_2,ST-2843,중곡3동_002_1,1,29,1220.89
3,20220901,0,0,ST-1251,가양1동_037_1,ST-2557,화곡6동_022_1,1,21,2357.82
4,20220901,0,0,ST-983,중곡1동_022_1,ST-106,성수2가1동_001_1,1,25,3800.00


In [31]:
october = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202210.csv', encoding = 'euc-kr')
print(october.shape)
october.head()

(8526697, 11)


,Unnamed: 0,기준일자,구분코드,기준시간대,시작대여소ID,시작대여소명,종료대여소ID,종료대여소명,전체건수,전체이용시간(분),전체이용거리(M)
0,0,20221027,0,0,ST-1287,하계1동_002_1,ST-2835,중계1동_024_1,1,10,1220.00
1,1,20221027,0,0,ST-1385,장위3동_005_1,ST-1280,월계1동_012_1,1,6,1470.00
2,2,20221027,0,0,ST-3078,신사동_021_3,ST-2688,압구정동_012_1,1,66,2075.21
3,3,20221027,0,0,ST-2823,신길6동_007_1,ST-668,구로3동_037_1,1,9,1512.96
4,4,20221027,0,0,ST-578,석관동_001_2,ST-2266,이문2동_015_1,1,5,1290.00


In [32]:
november = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202211.csv', encoding = 'euc-kr')
print(november.shape)
november.head()

(6838458, 8)


,Unnamed: 0,기준_날짜,기준_시간대,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리
0,0,20221130,0,ST-1093,ST-1093,1,0,0
1,1,20221130,0,ST-1409,ST-1409,1,0,0
2,2,20221130,0,ST-3072,ST-3072,1,1,0
3,3,20221130,5,ST-1722,ST-1722,1,0,0
4,4,20221130,5,ST-2235,ST-2235,1,0,0


In [33]:
december = pd.read_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202212.csv', encoding = 'euc-kr')
print(december.shape)
december.head()

(3497198, 8)


,Unnamed: 0,기준_날짜,기준_시간대,시작_대여소_ID,종료_대여소_ID,전체_건수,전체_이용_분,전체_이용_거리
0,0,20221225,0,ST-1407,ST-1407,2,0,0
1,1,20221225,0,ST-2119,ST-2119,1,0,0
2,2,20221225,0,ST-483,ST-483,1,0,30
3,3,20221225,5,ST-1320,ST-1320,1,0,0
4,4,20221225,5,ST-176,ST-176,1,1,30


In [ ]:
all_files = glob.glob('../ddarung_data/tpss_bcycl_od_statnhm/*.csv')
print(len(all_files))
all_files_data = []

for file in all_files:
    print(file)
    if(file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202205.csv' or 
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202211.csv' or 
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202210.csv' or 
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202204.csv' or
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202212.csv' or
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202206.csv' or
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202207.csv' or
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202203.csv' or
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202202.csv' or
       file == '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202201.csv'):
        data_frame = pd.read_csv(file, encoding = 'cp949')
    else:
        data_frame = pd.read_csv(file, encoding = 'utf8')
    
    all_files_data.append(data_frame)
    
all_files_data_concat = pd.concat(all_files_data, axis = 0, ignore_index = True)
print(all_files_data_concat.shape)
#all_files_data_concat.to_csv('../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202212.csv', encoding = 'euc-kr')

In [69]:
count = 1

print(janurary.shape[0])

for i in range(0, janurary.shape[0], 1000000):
    print(i)
    if(janurary.shape[0] > i+1000000):
        temp = janurary[i : i+1000000]
    else:
        temp = janurary[i : janurary.shape[0]]
    
    temp = temp.drop(columns = ['Unnamed: 0'])
    url = '../ddarung_data/tpss_bcycl_od_statnhm/tpss_bcycl_od_statnhm_202201/tpss_bcycl_od_statnhm_202201_%d.csv'%count
    temp.to_csv(url, encoding = 'euc-kr')
    count = count + 1

9290665
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
